In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random
import tensorflow as tf
import datetime
import struct
import time
from sklearn import linear_model
from sklearn import svm
import sklearn.model_selection
import sklearn.metrics
import sklearn.preprocessing

root_dir = "D:/Jupyter/";
logs_dir = root_dir + "Logs/"
data_dir = root_dir + 'Datasets/'

def mnist_read_imgs(fname):
    with open(fname, mode='rb') as f:
        (_, img_num, img_xsize, img_ysize) = struct.unpack('>IIII',f.read(4 * 4))
        data_img = np.fromfile(f, dtype=np.uint8).reshape(img_num, img_xsize, img_ysize)
    return data_img

def mnist_read_lbls(fname):
    with open(data_dir + 'MNIST/train-labels.idx1-ubyte', mode='rb') as f:
        (_, lab_num) = struct.unpack('>II', f.read(4 * 2))
        data_lab = np.fromfile(f, dtype=np.uint8)
    return data_lab

def minibatch(X, y, num=1000):
    inds = np.random.choice(range(X.shape[0]), size=num)
    return X[inds], y[inds]

In [2]:
src_X = mnist_read_imgs(data_dir+'MNIST/train-images.idx3-ubyte')
src_y = mnist_read_lbls(data_dir+'MNIST/train-labels.idx1-ubyte')

random_seed = 42
(dev_X, test_X, dev_y, test_y) = sklearn.model_selection.train_test_split(src_X, src_y, random_state=random_seed, test_size=0.2)
(train_X, valid_X, train_y, valid_y) = sklearn.model_selection.train_test_split(dev_X, dev_y, random_state=random_seed, test_size=0.2)

In [3]:
def mnist2d_transform_imgs(x):
    return x.reshape(x.shape[0], x.shape[1], x.shape[2], 1) / 255

def mnist2d_transform_lbls(y):
    return np.array([1.0*(y==i) for i in range(10)]).transpose()

In [4]:
(train2d_X, valid2d_X, test2d_X) = (mnist2d_transform_imgs(x) for x in (train_X, valid_X, test_X))
(train2d_y, valid2d_y, test2d_y) = (mnist2d_transform_lbls(y) for y in (train_y, valid_y, test_y))

In [5]:
def tf_conv2d(x, W):
  """conv2d returns a 2d convolution layer with full stride."""
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def tf_max_pool_2x2(x):
  """max_pool_2x2 downsamples a feature map by 2X."""
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def tf_weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)


def tf_bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [30]:
def deepnn(x):
  # First convolutional layer - maps one grayscale image to 32 feature maps.
    with tf.name_scope('conv1'):
        W_conv1 = tf_weight_variable([3, 3, 1, 15]) #3x3 15 instead 5x5 32
        b_conv1 = tf_bias_variable([15])
        h_conv1 = tf.nn.relu(tf_conv2d(x, W_conv1) + b_conv1)

  # Pooling layer - downsamples by 2X.
    with tf.name_scope('pool1'):
        h_pool1 = tf_max_pool_2x2(h_conv1)

  # Second convolutional layer -- maps 32 feature maps to 64.
    with tf.name_scope('conv2'):
        W_conv2 = tf_weight_variable([3, 3, 15, 15]) #3x3 15 instead 5x5 64
        b_conv2 = tf_bias_variable([15])
        h_conv2 = tf.nn.relu(tf_conv2d(h_pool1, W_conv2) + b_conv2)

  # Second pooling layer.
    with tf.name_scope('pool2'):
        h_pool2 = tf_max_pool_2x2(h_conv2)

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features.
    with tf.name_scope('fc1'):
        W_fc1 = tf_weight_variable([7 * 7 * 15, 100])
        b_fc1 = tf_bias_variable([100])

        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*15])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Map the 1024 features to 10 classes, one for each digit
    with tf.name_scope('fc2'):
        W_fc2 = tf_weight_variable([100, 10])
        b_fc2 = tf_bias_variable([10])

        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    return y_conv, keep_prob

In [31]:
tf.reset_default_graph()
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
log_dir = root_dir + 'Logs/' + dt_now + '-GOOGLE-TF'

#tfLR = tf.placeholder(shape=(),dtype=tf.float32)
tfInput = tf.placeholder(shape=(None,28,28,1),dtype=tf.float32)
tfLabels = tf.placeholder(shape=(None,10),dtype=tf.float32)

tfLogits, tfPKeep = deepnn(tfInput)

tfLoss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tfLabels, logits=tfLogits))

tfOutProb = tf.nn.softmax(tfLogits, name='OutputProbs')
tfAccuracy = 1.0-tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tfOutProb, axis=1),tf.argmax(tfLabels, axis=1)), dtype=tf.float32))
tfAccuracySummary = tf.summary.scalar('Accuracy', tfAccuracy)

#tfTrain = tf.train.RMSPropOptimizer(1e-4).minimize(tfLoss)
tfTrain = tf.train.AdamOptimizer(1e-4).minimize(tfLoss)


tffw = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [33]:
vX, vy = minibatch(valid2d_X, valid2d_y, num=5000)
valid_batch = {tfInput: vX, tfLabels: vy, tfPKeep: 1.0}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        #batch = mnist.train.next_batch(50)
        tX, ty = minibatch(train2d_X, train2d_y, num=50)
        tfTrain.run(feed_dict={tfInput: tX, tfLabels: ty, tfPKeep: 0.5})
        if i % 100 == 0:
            print(tfLoss.eval(feed_dict={tfInput: tX, tfLabels: ty, tfPKeep: 1.0}))
            valid_acc_str = tfAccuracySummary.eval(feed_dict=valid_batch)
            tffw.add_summary(valid_acc_str, i / 100)
        #print('step %d, training accuracy %g' % (i, train_accuracy))

2.2865
2.20936
1.99161
1.65117
1.1338
0.838299
0.71089
0.416916
0.488635
0.507289
0.402624
0.420364
0.255835
0.388277
0.234811
0.480529
0.2737
0.337841
0.287743
0.139425
0.242976
0.263951
0.467871
0.192354
0.205917
0.168718
0.24461
0.0969818
0.0849472
0.336965
0.131955
0.211244
0.149919
0.0991082
0.0807481
0.115081


KeyboardInterrupt: 